<a href="https://colab.research.google.com/github/Iresh-github/GBM-analysis/blob/main/PI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Research with Sam/PreProcess_data_2_train.csv')
data.head()

,VOLUME_ET,VOLUME_NET,VOLUME_ED,VOLUME_TC,VOLUME_WT,VOLUME_BRAIN,VOLUME_ET_OVER_NET,VOLUME_ET_OVER_ED,VOLUME_NET_OVER_ED,VOLUME_ET_over_TC,...,TEXTURE_NGTDM_NET_FLAIR_Complexity,TEXTURE_NGTDM_NET_FLAIR_Strength,TGM_p1,TGM_dw,TGM_Cog_X_1,TGM_Cog_Y_1,TGM_Cog_Z_1,TGM_T_1,Age,Survival_cat
0,1662,384,36268,2046,38314,1469432,4.328125,0.045826,0.010588,0.81232,...,2203.2034,61.3293,8.00000,7.500000e-07,0.178609,0.096256,0.052741,2.0000,56.208219,HIGH
1,4362,4349,15723,8711,24434,1295721,1.002989,0.277428,0.276601,0.50075,...,1440.4285,3.5999,3.31250,1.000000e-09,0.077619,0.122900,0.094336,91.4736,61.526027,MID
2,33404,48612,45798,82016,127814,1425843,0.687155,0.729377,1.061444,0.40729,...,1147.5177,0.8048,5.78125,1.000000e-09,0.132283,0.116006,0.096035,272.4290,18.975342,HIGH
3,12114,7587,34086,19701,53787,1403429,1.596679,0.355395,0.222584,0.61489,...,3320.1690,4.7336,3.87500,1.000000e-09,0.100415,0.088249,0.096470,128.4680,33.887671,MID
4,34538,7137,65653,41675,107328,1365237,4.839288,0.526069,0.108708,0.82875,...,1834.1052,2.4532,1.00000,5.730000e-08,0.106184,0.131952,0.096894,240.7780,54.986301,LOW


In [7]:
from google.colab import files
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

In [8]:
# Separate features and target variable
x = data.drop('Survival_cat', axis=1)
y = data['Survival_cat']

In [9]:
x

,VOLUME_ET,VOLUME_NET,VOLUME_ED,VOLUME_TC,VOLUME_WT,VOLUME_BRAIN,VOLUME_ET_OVER_NET,VOLUME_ET_OVER_ED,VOLUME_NET_OVER_ED,VOLUME_ET_over_TC,...,TEXTURE_NGTDM_NET_FLAIR_Busyness,TEXTURE_NGTDM_NET_FLAIR_Complexity,TEXTURE_NGTDM_NET_FLAIR_Strength,TGM_p1,TGM_dw,TGM_Cog_X_1,TGM_Cog_Y_1,TGM_Cog_Z_1,TGM_T_1,Age
0,1662,384,36268,2046,38314,1469432,4.328125,0.045826,0.010588,0.81232,...,0.021897,2203.2034,61.32930,8.00000,7.500000e-07,0.178609,0.096256,0.052741,2.0000,56.208219
1,4362,4349,15723,8711,24434,1295721,1.002989,0.277428,0.276601,0.50075,...,0.189980,1440.4285,3.59990,3.31250,1.000000e-09,0.077619,0.122900,0.094336,91.4736,61.526027
2,33404,48612,45798,82016,127814,1425843,0.687155,0.729377,1.061444,0.40729,...,1.068800,1147.5177,0.80480,5.78125,1.000000e-09,0.132283,0.116006,0.096035,272.4290,18.975342
3,12114,7587,34086,19701,53787,1403429,1.596679,0.355395,0.222584,0.61489,...,0.206600,3320.1690,4.73360,3.87500,1.000000e-09,0.100415,0.088249,0.096470,128.4680,33.887671
4,34538,7137,65653,41675,107328,1365237,4.839288,0.526069,0.108708,0.82875,...,0.413750,1834.1052,2.45320,1.00000,5.730000e-08,0.106184,0.131952,0.096894,240.7780,54.986301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,17626,3274,62021,20900,82921,1409680,5.383629,0.284194,0.052789,0.84335,...,0.075237,2618.3822,13.30410,1.00000,1.080000e-07,0.166801,0.092181,0.087228,94.3296,57.345205
72,44540,28928,130833,73468,204301,1653413,1.539685,0.340434,0.221106,0.60625,...,0.628460,1648.6025,1.38760,1.50000,2.160000e-08,0.146336,0.075623,0.074336,288.9860,63.898630
73,14699,11694,17488,26393,43881,1416517,1.256969,0.840519,0.668687,0.55693,...,0.325090,1956.7727,3.26780,1.00000,1.000000e-09,0.157037,0.149968,0.079077,151.8750,64.860274
74,58556,9087,88963,67643,156606,1466297,6.443931,0.658206,0.102144,0.86566,...,0.515520,1276.9291,2.26680,1.00000,2.440000e-08,0.138023,0.135957,0.120478,240.2820,66.479452


In [10]:
y

0     HIGH
1      MID
2     HIGH
3      MID
4      LOW
      ... 
71     MID
72    HIGH
73     LOW
74     LOW
75     LOW
Name: Survival_cat, Length: 76, dtype: object

In [11]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
y_test

4      LOW
35    HIGH
10     LOW
0     HIGH
45     LOW
47     MID
65     LOW
53     MID
50    HIGH
28     MID
67    HIGH
70     LOW
18    HIGH
12    HIGH
58    HIGH
33    HIGH
Name: Survival_cat, dtype: object

In [13]:
# Create and fit a RandomForestClassifier (or any other model of your choice)
model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [14]:
# Compute permutation importance
perm_importance = permutation_importance(model, x_test, y_test, n_repeats=30, random_state=42)
perm_importance


{'importances_mean': array([0.        , 0.        , 0.03125   , 0.        , 0.        ,
        0.04583333, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00208333, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.05      , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.05416667, 0.        ,
        0.        , 0.        , 0.        , 0.05833333, 0.        ,
        0.        , 0.        , 0.        , 0.025     , 0.03333333,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.03958333, 0.        , 0.        ,
        0.        , 0.02083333, 0.        , 0.        , 0.        ,
        0.        , 0.      

In [16]:
# Extract feature importances
importances = perm_importance.importances_mean
importances

array([0.        , 0.        , 0.03125   , 0.        , 0.        ,
       0.04583333, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00208333, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.05      , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.05416667, 0.        ,
       0.        , 0.        , 0.        , 0.05833333, 0.        ,
       0.        , 0.        , 0.        , 0.025     , 0.03333333,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.03958333, 0.        , 0.        ,
       0.        , 0.02083333, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [17]:
feature_importance_df = pd.DataFrame({'Feature': x.columns, 'Importance': importances})
sorted_feature_importance = feature_importance_df.sort_values(by='Importance', ascending=False)
print(sorted_feature_importance)

                                 Feature  Importance
607           TEXTURE_GLSZM_NET_T1_SZHGE    0.060417
317        TEXTURE_GLCM_NET_FLAIR_Energy    0.060417
448            TEXTURE_GLRLM_NET_T1_LGRE    0.060417
304       TEXTURE_GLCM_NET_T1_SumAverage    0.060417
307  TEXTURE_GLCM_NET_T1_AutoCorrelation    0.058333
..                                   ...         ...
265           TEXTURE_GLCM_ED_T1_Entropy    0.000000
266       TEXTURE_GLCM_ED_T1_Homogeneity    0.000000
267       TEXTURE_GLCM_ED_T1_Correlation    0.000000
268        TEXTURE_GLCM_ED_T1_SumAverage    0.000000
352              TEXTURE_GLRLM_ET_T2_SRE    0.000000

[705 rows x 2 columns]


In [18]:
##o print all the first 50 values in the DataFrame generated
for index, row in sorted_feature_importance.head(50).iterrows():
    print("Feature:", row['Feature'])
    print("Importance:", row['Importance'])
    print()  # Print an empty line for better readability

Feature: TEXTURE_GLSZM_NET_T1_SZHGE
Importance: 0.06041666666666667

Feature: TEXTURE_GLCM_NET_FLAIR_Energy
Importance: 0.06041666666666667

Feature: TEXTURE_GLRLM_NET_T1_LGRE
Importance: 0.06041666666666667

Feature: TEXTURE_GLCM_NET_T1_SumAverage
Importance: 0.06041666666666667

Feature: TEXTURE_GLCM_NET_T1_AutoCorrelation
Importance: 0.058333333333333334

Feature: HISTO_NET_FLAIR_Bin3
Importance: 0.058333333333333334

Feature: TEXTURE_GLCM_ET_T1_Dissimilarity
Importance: 0.058333333333333334

Feature: HISTO_ET_T1Gd_Bin2
Importance: 0.058333333333333334

Feature: TEXTURE_GLCM_NET_T1Gd_Dissimilarity
Importance: 0.05625

Feature: TEXTURE_GLCM_NET_T2_Correlation
Importance: 0.05625

Feature: INTENSITY_Mean_ED_T2
Importance: 0.05416666666666667

Feature: TEXTURE_GLCM_ET_FLAIR_Dissimilarity
Importance: 0.05416666666666667

Feature: HISTO_ET_T2_Bin4
Importance: 0.052083333333333336

Feature: HISTO_NET_FLAIR_Bin6
Importance: 0.052083333333333336

Feature: TEXTURE_GLRLM_NET_FLAIR_RLV
Importa